In [1]:
# --- start make file run from another dir ---
#
# Note: File should be run from XKE root dir. E.g.:
#
#       $ cd Projects/XKE
#       $ python examples/emb_grid_search/grid_search_TransE_FB13.py
#
import os, sys
file_ = os.path.join(os.path.abspath(sys.path[0])) # use this when inside a jupyter notebook
# file_ = __file__ # use this when running from command line
xke_root = os.path.abspath(os.path.join(file_, "../"))
sys.path.insert(0, xke_root)
# --- end make file run from another dir ---

In [2]:
from sfe.sfe import Graph, SFE
import pandas as pd
import numpy as np
import os, time

def debug_print(list_of_els):
    """Prints a list of elements using their string method."""
    l = []
    for n in list_of_els:
        l.append(n.__str__())
    print(l)
    
def print_node_seqs(node_seqs):
    """Prints a node sequences dict."""
    for node,seqs in node_seqs.iteritems():
        print "{}:".format(node)
        for seq in seqs: debug_print(seq)
        print

# FB13

In [4]:
dataset_path = '../benchmarks/FB13'

train2id = pd.read_csv(dataset_path + '/train2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
valid2id = pd.read_csv(dataset_path + '/valid2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
test2id = pd.read_csv(dataset_path + '/test2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])

from tools import dataset_tools
entity2id, id2entity     = dataset_tools.read_name2id_file(dataset_path + '/entity2id.txt')
relation2id, id2relation = dataset_tools.read_name2id_file(dataset_path + '/relation2id.txt')

train = pd.read_csv(dataset_path + '/train.txt', sep='\t', skiprows=0, names=['head', 'relation', 'tail'])
valid = pd.read_csv(dataset_path + '/valid.txt', sep='\t', skiprows=0, names=['head', 'relation', 'tail', 'label'])
test = pd.read_csv(dataset_path + '/test.txt', sep='\t', skiprows=0, names=['head', 'relation', 'tail', 'label'])

valid_pos = valid.loc[valid['label'] == 1]

In [5]:
start_time = time.time()
g = Graph()
g.partial_build_from_df(train)
g.partial_build_from_df(valid_pos)
print("Elapsed time: {}".format(time.time() - start_time))

Elapsed time: 44.326305151


In [9]:
sfe = SFE(g, max_depth=4, max_fan_out=1500)

In [10]:
%%time
# paths = sfe.search_paths('camilo_jose_cela', 'spain', max_depth=2)
paths = sfe.search_paths('camilo_jose_cela', 'male')

time get nodes: 5.96046447754e-06
time to find node sequences: 23.0675179958
time to merge node sequences: 0.911879062653
time to get paths: 2.61148285866
CPU times: user 27.4 s, sys: 398 ms, total: 27.8 s
Wall time: 27.6 s


In [11]:
lpaths = {}
for path in paths:
    lpaths[len(path)] = lpaths.get(len(path), []) + [path]
for idx, ps in lpaths.iteritems():
    print "Paths of length: ", idx
    for p in ps: print p
    print
print "\nNumber of paths: {}\n\n".format(len(paths))

Paths of length:  1
('gender',)

Paths of length:  3
('nationality', '_place_of_death', 'gender')
('ethnicity', '_ethnicity', 'gender')
('nationality', '_institution', 'gender')
('place_of_death', '_location', 'gender')
('profession', '_profession', 'gender')
('nationality', '_location', 'gender')
('place_of_death', '_place_of_birth', 'gender')
('place_of_death', '_place_of_death', 'gender')
('nationality', '_place_of_birth', 'gender')
('nationality', '_nationality', 'gender')

Paths of length:  4
('nationality', '_nationality', '_spouse', 'gender')
('profession', '_profession', '_spouse', 'gender')
('nationality', '_nationality', 'spouse', 'gender')
('place_of_death', '_place_of_death', '_parents', 'gender')
('nationality', '_place_of_birth', '_children', 'gender')
('ethnicity', '_ethnicity', 'parents', 'gender')
('place_of_death', '_location', '_parents', 'gender')
('ethnicity', '_ethnicity', '_children', 'gender')
('nationality', '_institution', '_children', 'gender')
('place_of_dea

In [11]:
train[1230:1231]

,head,relation,tail
1230,beals_wright,gender,male


In [12]:
df = train[1230:1231]
feats = sfe.generate_features(df, max_depth=2)

TypeError: generate_features() got an unexpected keyword argument 'max_depth'

In [ ]:
%%time
feats = next(feats)

In [8]:
# for idx,f_ in enumerate(feats):
#     f = f_[1]
#     row = df.iloc[idx]
#     print(row['head'], row['relation'], row['tail'], f)
#     print "\nNumber of features: {}\n\n".format(len(f))

Experimenting with IDs instead of names: it did not help much.

In [ ]:
start_time = time.time()
g2 = Graph()
g2.partial_build_from_df(train2id)
g2.partial_build_from_df(valid2id)
print("Elapsed time: {}".format(time.time() - start_time))

In [ ]:
sfe2 = SFE(g2)

In [ ]:
%%timeit
df = train2id[1230:1231]
feats = sfe2.generate_features(df, max_depth=2)
feats = next(feats)

In [ ]:
%%timeit
paths = sfe2.search_paths(entity2id['beals_wright'], entity2id['male'], max_depth=2)

In [ ]:
for idx,f in feats:
    print idx
    row = df.iloc[idx]
    print(row['head'], row['relation'], row['tail'], f)

In [ ]:
head, rel, tail, paths = (29639, 6, 67575, [['10', '_5'], ['_9', '_5'], ['9', '_5'], ['_10', '_5'], ['2', '_2', '_5'], ['_11', '_5'], ['11', '_5']])

In [ ]:
print id2entity[head]
print id2relation[rel]
print id2entity[tail]
for path in paths:
    p = []
    for r in path:
        relation_id = int(r.replace('_', ''))
        relation_label = id2relation[relation_id]
        p.append(r.replace(str(relation_id), relation_label))
    print p

PRA results for `camilo_jose_cela` `nationality` `spain`:

```
-gender-_gender-place_of_birth-,1.0 -#-
-gender-_gender-location-,1.0 -#-
-gender-_gender-place_of_death-,1.0 -#-
-profession-_profession-place_of_birth-,1.0 -#-
-profession-_profession-place_of_death-,1.0 -#-
-place_of_death-_place_of_death-place_of_birth-,1.0 -#-
-place_of_death-_location-place_of_birth-,1.0 -#-
-place_of_death-_place_of_birth-place_of_death-,1.0 -#-
-place_of_death-_place_of_death-location-,1.0 -#-
-place_of_death-_location-location-,1.0 -#-
-profession-_profession-location-,1.0 -#-
-gender-_gender-institution-,1.0
```

In [ ]:
paths = sfe.search_paths('spain', 'camilo_jose_cela', max_depth=2)
for path in paths:
    path_strings = []
    for r in path:
        path_strings.append(r.__str__())
    print path_strings

In [ ]:
node = sfe.graph.get_node('menasseh_ben_israel')

In [ ]:
for n in node.neighbors: print n

In [ ]:
node = sfe.graph.get_node('spain')
for n in node.neighbors: print n

In [ ]:
sfe.graph.get_node('richard_baxter') in node.neighbors

In [ ]:
# run for one triple
start_time = time.time()
paths = sfe.extract_features(0, 67393, 2)
for p in paths:
    print "-----------path-----------"
    for e in p:
        print e.__str__()
print("Elapsed time: {}".format(time.time() - start_time))

In [ ]:
# run for n triples
start_time = time.time()
res = []
for idx,row in train[:100].iterrows():
    res.append(sfe.extract_features(row['head'], row['tail'], 2))
print("Elapsed time: {}".format(time.time() - start_time))

In [ ]:
with open('output_test.txt', 'w') as f:
    for row in res:
        for seq in row:
            for edge in seq:
                f.write(edge.__str__())
                f.write(',')
            f.write("\n")

In [ ]:
!cat output_test.txt

---

# NELL186

In [97]:
dataset_path = './benchmarks/NELL186'

train2id = pd.read_csv(dataset_path + '/train2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
valid2id = pd.read_csv(dataset_path + '/valid2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
test2id = pd.read_csv(dataset_path + '/test2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])

from tools import dataset_tools
entity2id, id2entity     = dataset_tools.read_name2id_file(dataset_path + '/entity2id.txt')
relation2id, id2relation = dataset_tools.read_name2id_file(dataset_path + '/relation2id.txt')

train = pd.read_csv(dataset_path + '/train.txt', sep='\t', skiprows=0, names=['head', 'relation', 'tail'])
valid = pd.read_csv(dataset_path + '/valid.txt', sep='\t', skiprows=0, names=['head', 'relation', 'tail', 'label'])
test = pd.read_csv(dataset_path + '/test.txt', sep='\t', skiprows=0, names=['head', 'relation', 'tail', 'label'])

valid_pos = valid.loc[valid['label'] == 1]

In [98]:
start_time = time.time()
g = Graph()
g.partial_build_from_df(train)
g.partial_build_from_df(valid_pos)
print("Elapsed time: {}".format(time.time() - start_time))

Elapsed time: 5.3636341095


In [99]:
sfe = SFE(g)

In [100]:
train.head()

,head,relation,tail
0,concept:academicfield:applied_science,concept:academicfieldsuchasacademicfield,concept:academicfield:engineering_technology
1,concept:academicfield:applied_science,concept:academicfieldsuchasacademicfield,concept:academicfield:engineering
2,concept:academicfield:applied_science,concept:academicfieldsuchasacademicfield,concept:academicfield:technology
3,concept:academicfield:applied_science,concept:academicfieldsuchasacademicfield,concept:academicfield:management
4,concept:academicfield:architecture,concept:academicprogramatuniversity,concept:university:ball_state


## Find paths between an instance (node pair)

In [101]:
%%time
paths = sfe.search_paths('concept:academicfield:applied_science', 'concept:academicfield:engineering_technology', max_depth=2)

time get nodes: 8.10623168945e-06
time to find node sequences: 0.000958919525146
time to merge node sequences: 0.00128316879272
time to get paths: 0.000714063644409
CPU times: user 4.72 ms, sys: 39 µs, total: 4.75 ms
Wall time: 4.22 ms


In [102]:
lpaths = {}
for path in paths:
    lpaths[len(path)] = lpaths.get(len(path), []) + [path]
for idx, ps in lpaths.iteritems():
    print "Paths of length: ", idx
    for p in ps: print p
    print
print "\nNumber of paths: {}\n\n".format(len(paths))

Paths of length:  1
('concept:academicfieldsuchasacademicfield',)

Paths of length:  2
('concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield')
('concept:academicfieldsuchasacademicfield', 'concept:academicfieldsuchasacademicfield')
('_concept:academicfieldsuchasacademicfield', 'concept:academicfieldsuchasacademicfield')

Paths of length:  3
('_concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield')
('concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield', 'concept:academicfieldsuchasacademicfield')
('concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield')
('concept:academicfieldsuchasacademicfield', 'concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicfield')
('_concept:academicfieldsuchasacademicfield', '_concept:academicfieldsuchasacademicf

## Try to run SFE for the entire dataset

In [103]:
valid.head()

,head,relation,tail,label
0,concept:academicfield:arts,concept:academicfieldsuchasacademicfield,concept:academicfield:architecture,1
1,concept:academicfield:arts,concept:academicfieldsuchasacademicfield,concept:governmentorganization:u_s__senate,-1
2,concept:academicfield:arts,concept:academicfieldsuchasacademicfield,concept:academicfield:technology,1
3,concept:athlete:bauer,concept:academicfieldsuchasacademicfield,concept:academicfield:technology,-1
4,concept:academicfield:biophysics,concept:academicfieldsuchasacademicfield,concept:academicfield:biochemistry,1


In [104]:
valid.label.unique()

array([ 1, -1])

In [105]:
len(valid.sort_values(by=['head', 'tail']))

10000

In [106]:
%%time
res = sfe.generate_features(valid, max_depth=2, batch_size=999999)

CPU times: user 837 µs, sys: 55 µs, total: 892 µs
Wall time: 908 µs


In [107]:
%%time
boo = next(res)

time get nodes: 1.19209289551e-05
time to find node sequences: 0.00314211845398
time to merge node sequences: 0.000387907028198
time to get paths: 1.38282775879e-05
time get nodes: 3.09944152832e-06
time to find node sequences: 0.000830173492432
time to merge node sequences: 0.00256180763245
time to get paths: 0.00284385681152
time get nodes: 5.00679016113e-06
time to find node sequences: 0.00121593475342
time to merge node sequences: 0.000476121902466
time to get paths: 3.88622283936e-05
time get nodes: 5.00679016113e-06
time to find node sequences: 0.000226974487305
time to merge node sequences: 0.000116109848022
time to get paths: 0.000193119049072
time get nodes: 5.96046447754e-06
time to find node sequences: 0.000592947006226
time to merge node sequences: 0.00189304351807
time to get paths: 0.00170302391052
time get nodes: 5.96046447754e-06
time to find node sequences: 0.000898838043213
time to merge node sequences: 0.00263810157776
time to get paths: 0.00221681594849
time get nod

time get nodes: 3.09944152832e-06
time to find node sequences: 0.0010449886322
time to merge node sequences: 0.000807046890259
time to get paths: 0.000169992446899
time get nodes: 4.05311584473e-06
time to find node sequences: 0.0020809173584
time to merge node sequences: 0.0049569606781
time to get paths: 0.00419521331787
time get nodes: 4.05311584473e-06
time to find node sequences: 0.00163912773132
time to merge node sequences: 0.00032901763916
time to get paths: 4.88758087158e-05
time get nodes: 3.09944152832e-06
time to find node sequences: 0.00703120231628
time to merge node sequences: 0.000190019607544
time to get paths: 1.31130218506e-05
time get nodes: 7.15255737305e-06
time to find node sequences: 0.000645160675049
time to merge node sequences: 0.00188112258911
time to get paths: 0.00160717964172
time get nodes: 3.09944152832e-06
time to find node sequences: 0.000725030899048
time to merge node sequences: 0.00204086303711
time to get paths: 0.00136399269104
time get nodes: 3.

time get nodes: 3.09944152832e-06
time to find node sequences: 0.000352144241333
time to merge node sequences: 9.98973846436e-05
time to get paths: 4.38690185547e-05
time get nodes: 3.81469726562e-06
time to find node sequences: 0.00271487236023
time to merge node sequences: 0.00262999534607
time to get paths: 0.00415897369385
time get nodes: 5.96046447754e-06
time to find node sequences: 0.0100979804993
time to merge node sequences: 0.000386953353882
time to get paths: 1.90734863281e-05
time get nodes: 5.00679016113e-06
time to find node sequences: 0.00253510475159
time to merge node sequences: 0.00829601287842
time to get paths: 0.00756096839905
time get nodes: 8.10623168945e-06
time to find node sequences: 0.00190186500549
time to merge node sequences: 0.00369501113892
time to get paths: 0.00624489784241
time get nodes: 6.91413879395e-06
time to find node sequences: 0.00138115882874
time to merge node sequences: 0.00025200843811
time to get paths: 0.000140905380249
time get nodes: 4

time to get paths: 1.38282775879e-05
time get nodes: 1.90734863281e-06
time to find node sequences: 0.000128030776978
time to merge node sequences: 3.09944152832e-06
time to get paths: 9.53674316406e-07
time get nodes: 3.81469726562e-06
time to find node sequences: 0.000102043151855
time to merge node sequences: 9.05990600586e-06
time to get paths: 1.90734863281e-06
time get nodes: 2.14576721191e-06
time to find node sequences: 8.60691070557e-05
time to merge node sequences: 1.62124633789e-05
time to get paths: 1.21593475342e-05
time get nodes: 2.14576721191e-06
time to find node sequences: 0.000643014907837
time to merge node sequences: 3.50475311279e-05
time to get paths: 1.00135803223e-05
time get nodes: 3.09944152832e-06
time to find node sequences: 0.000184059143066
time to merge node sequences: 1.50203704834e-05
time to get paths: 2.14576721191e-06
time get nodes: 5.00679016113e-06
time to find node sequences: 0.000489950180054
time to merge node sequences: 1.71661376953e-05
time

time get nodes: 4.05311584473e-06
time to find node sequences: 0.000387907028198
time to merge node sequences: 0.000237941741943
time to get paths: 0.000131845474243
time get nodes: 2.86102294922e-06
time to find node sequences: 0.000198125839233
time to merge node sequences: 0.000162124633789
time to get paths: 6.00814819336e-05
time get nodes: 3.09944152832e-06
time to find node sequences: 0.000471115112305
time to merge node sequences: 0.000293970108032
time to get paths: 0.000184059143066
time get nodes: 5.00679016113e-06
time to find node sequences: 0.000109910964966
time to merge node sequences: 7.79628753662e-05
time to get paths: 5.00679016113e-06
time get nodes: 2.86102294922e-06
time to find node sequences: 0.000837087631226
time to merge node sequences: 0.000545024871826
time to get paths: 0.00032114982605
time get nodes: 3.81469726562e-06
time to find node sequences: 0.000395059585571
time to merge node sequences: 0.000481843948364
time to get paths: 0.000555038452148
time 

...

time to merge node sequences: 0.0028350353241
time to get paths: 0.000560998916626
time get nodes: 5.96046447754e-06
time to find node sequences: 0.000643968582153
time to merge node sequences: 0.000529050827026
time to get paths: 5.00679016113e-06
time get nodes: 5.96046447754e-06
time to find node sequences: 0.002121925354
time to merge node sequences: 0.00145411491394
time to get paths: 0.000427961349487
time get nodes: 5.00679016113e-06
time to find node sequences: 0.000466108322144
time to merge node sequences: 0.00023078918457
time to get paths: 1.90734863281e-05
time get nodes: 3.81469726562e-06
time to find node sequences: 0.000434875488281
time to merge node sequences: 0.000370025634766
time to get paths: 2.86102294922e-06
time get nodes: 5.96046447754e-06
time to find node sequences: 0.00142383575439
time to merge node sequences: 0.000293016433716
time to get paths: 2.19345092773e-05
time get nodes: 5.96046447754e-06
time to find node sequences: 0.0011031627655
time to merge 

time to get paths: 2.14576721191e-06
time get nodes: 2.86102294922e-06
time to find node sequences: 0.000452995300293
time to merge node sequences: 0.000574111938477
time to get paths: 0.000242948532104
time get nodes: 5.00679016113e-06
time to find node sequences: 0.00234293937683
time to merge node sequences: 0.000157117843628
time to get paths: 3.09944152832e-06
time get nodes: 3.09944152832e-06
time to find node sequences: 0.000178098678589
time to merge node sequences: 0.000118970870972
time to get paths: 2.86102294922e-06
time get nodes: 3.09944152832e-06
time to find node sequences: 0.00211095809937
time to merge node sequences: 0.00202703475952
time to get paths: 0.000562906265259
time get nodes: 5.96046447754e-06
time to find node sequences: 0.000797986984253
time to merge node sequences: 0.000227928161621
time to get paths: 1.90734863281e-06
time get nodes: 4.05311584473e-06
time to find node sequences: 0.001060962677
time to merge node sequences: 0.0010039806366
time to get 

time get nodes: 5.00679016113e-06
time to find node sequences: 0.000837087631226
time to merge node sequences: 0.00049901008606
time to get paths: 9.05990600586e-06
time get nodes: 2.86102294922e-06
time to find node sequences: 0.000910997390747
time to merge node sequences: 0.000652074813843
time to get paths: 0.000230073928833
time get nodes: 5.00679016113e-06
time to find node sequences: 5.60283660889e-05
time to merge node sequences: 1.69277191162e-05
time to get paths: 8.82148742676e-06
time get nodes: 2.86102294922e-06
time to find node sequences: 0.00151205062866
time to merge node sequences: 0.000492095947266
time to get paths: 1.59740447998e-05
time get nodes: 2.86102294922e-06
time to find node sequences: 0.000622987747192
time to merge node sequences: 0.000178098678589
time to get paths: 7.29560852051e-05
time get nodes: 3.09944152832e-06
time to find node sequences: 0.00130796432495
time to merge node sequences: 1.09672546387e-05
time to get paths: 1.19209289551e-06
time ge

time to get paths: 2.14576721191e-06
time get nodes: 4.05311584473e-06
time to find node sequences: 0.00151801109314
time to merge node sequences: 1.90734863281e-05
time to get paths: 2.14576721191e-06
time get nodes: 2.86102294922e-06
time to find node sequences: 0.000624895095825
time to merge node sequences: 1.78813934326e-05
time to get paths: 2.14576721191e-06
time get nodes: 5.96046447754e-06
time to find node sequences: 0.00119400024414
time to merge node sequences: 7.39097595215e-05
time to get paths: 2.86102294922e-06
time get nodes: 3.09944152832e-06
time to find node sequences: 0.00116991996765
time to merge node sequences: 0.00106000900269
time to get paths: 0.000834226608276
time get nodes: 3.09944152832e-06
time to find node sequences: 0.000625133514404
time to merge node sequences: 5.60283660889e-05
time to get paths: 2.14576721191e-06
time get nodes: 4.05311584473e-06
time to find node sequences: 0.000898122787476
time to merge node sequences: 1.90734863281e-05
time to 

time to merge node sequences: 5.00679016113e-05
time to get paths: 1.90734863281e-06
time get nodes: 3.81469726562e-06
time to find node sequences: 0.000442981719971
time to merge node sequences: 0.000355005264282
time to get paths: 1.90734863281e-06
time get nodes: 2.14576721191e-06
time to find node sequences: 0.00359702110291
time to merge node sequences: 0.0021550655365
time to get paths: 0.00150895118713
time get nodes: 5.00679016113e-06
time to find node sequences: 0.00186610221863
time to merge node sequences: 0.000990867614746
time to get paths: 0.000545978546143
time get nodes: 5.00679016113e-06
time to find node sequences: 0.00168704986572
time to merge node sequences: 0.00037693977356
time to get paths: 3.09944152832e-06
time get nodes: 3.09944152832e-06
time to find node sequences: 0.0036997795105
time to merge node sequences: 0.00186395645142
time to get paths: 0.00156903266907
time get nodes: 4.05311584473e-06
time to find node sequences: 0.00240111351013
time to merge no

In [109]:
len(boo)

10000

In [82]:
len(boo)

5000

# Debug python

In [ ]:
l = [1,2,3,4,5,6]

In [ ]:
l.remove(3)

In [ ]:
l

In [ ]:
s = {1,2,3,4}

In [ ]:
s - {4}

In [ ]:
s

In [ ]:
from itertools import product
l = [[1], [5,90,84]]
res = product(*l)
s = set()
s.update(res)
s

In [37]:
import math
float('inf') > 10

True

# Investigating difference in:

1. Max Depth = 2 and Max Fan Out = Infinite
2. Max Depth = 4 and Max Fan Out = 1000

In [5]:
sfe.max_depth = 2
sfe.max_fan_out = 99999999999
paths2inf = sfe.search_paths('camilo_jose_cela', 'male')
paths2inf

time get nodes: 5.96046447754e-06
time to find node sequences: 4.66664481163
time to merge node sequences: 0.186609983444
time to get paths: 0.025260925293


{('ethnicity', '_ethnicity', '_children', 'gender'),
 ('ethnicity', '_ethnicity', '_parents', 'gender'),
 ('ethnicity', '_ethnicity', '_spouse', 'gender'),
 ('ethnicity', '_ethnicity', 'children', 'gender'),
 ('ethnicity', '_ethnicity', 'gender'),
 ('ethnicity', '_ethnicity', 'parents', 'gender'),
 ('ethnicity', '_ethnicity', 'spouse', 'gender'),
 ('gender',),
 ('nationality', '_institution', '_children', 'gender'),
 ('nationality', '_institution', 'gender'),
 ('nationality', '_institution', 'parents', 'gender'),
 ('nationality', '_location', 'gender'),
 ('nationality', '_nationality', '_children', 'gender'),
 ('nationality', '_nationality', '_parents', 'gender'),
 ('nationality', '_nationality', '_place_of_death', 'gender'),
 ('nationality', '_nationality', '_spouse', 'gender'),
 ('nationality', '_nationality', 'children', 'gender'),
 ('nationality', '_nationality', 'gender'),
 ('nationality', '_nationality', 'parents', 'gender'),
 ('nationality', '_nationality', 'spouse', 'gender'),


In [23]:
sfe.max_depth = 4
sfe.max_fan_out = 1494
paths4none = sfe.search_paths('camilo_jose_cela', 'male')
paths4none

time get nodes: 5.96046447754e-06
time to find node sequences: 6.93193221092
time to merge node sequences: 0.368894815445
time to get paths: 0.00637888908386


{('ethnicity', '_ethnicity', '_children', 'gender'),
 ('ethnicity', '_ethnicity', '_parents', 'gender'),
 ('ethnicity', '_ethnicity', '_spouse', 'gender'),
 ('ethnicity', '_ethnicity', 'children', 'gender'),
 ('ethnicity', '_ethnicity', 'gender'),
 ('ethnicity', '_ethnicity', 'parents', 'gender'),
 ('ethnicity', '_ethnicity', 'spouse', 'gender'),
 ('gender',),
 ('nationality', '_institution', '_children', 'gender'),
 ('nationality', '_institution', 'gender'),
 ('nationality', '_institution', 'parents', 'gender'),
 ('nationality', '_location', 'gender'),
 ('nationality', '_nationality', '_children', 'gender'),
 ('nationality', '_nationality', '_parents', 'gender'),
 ('nationality', '_nationality', '_place_of_death', 'gender'),
 ('nationality', '_nationality', '_spouse', 'gender'),
 ('nationality', '_nationality', 'children', 'gender'),
 ('nationality', '_nationality', 'gender'),
 ('nationality', '_nationality', 'parents', 'gender'),
 ('nationality', '_nationality', 'spouse', 'gender'),


In [24]:
paths2inf.difference(paths4none)

{('profession', '_profession', '_children', 'gender'),
 ('profession', '_profession', '_parents', 'gender'),
 ('profession', '_profession', '_spouse', 'gender'),
 ('profession', '_profession', 'children', 'gender'),
 ('profession', '_profession', 'gender'),
 ('profession', '_profession', 'parents', 'gender'),
 ('profession', '_profession', 'spouse', 'gender')}

In [8]:
camilo_jose_celta = g.get_node('camilo_jose_cela')

In [9]:
debug_print(camilo_jose_celta.neighbors)

['Node(novelist)', 'Node(writer)', 'Node(galician_people)', 'Node(male)', 'Node(madrid)', 'Node(spain)']


In [20]:
print len(g.get_node('writer').neighbors)
print g.get_node('writer').fan_out
print len(g.get_node('male').neighbors)
print g.get_node('male').fan_out
print len(g.get_node('novelist').neighbors)
print g.get_node('novelist').fan_out

4404
4404
53662
53662
1495
1495


In [14]:
sfe.bfs_node_seqs(sfe.graph.get_node('male'))

{}

In [19]:
camilo_jose_celta.neighbor2edgesstr[sfe.graph.get_node('novelist')]

['profession']